In [40]:
import os
import yfinance as yf
import pandas as pd
import datetime as dt
from pandas.tseries.offsets import MonthEnd
import feather
import pyarrow.parquet as pq
import pyarrow as pa


In [41]:
df = pd.read_csv('sp100.csv')
tickers =  [i.replace('.','-') for i in df.Symbol.to_list()]

file = 'datasets/sp100.parquet'
if not os.path.isfile(file):
    df = yf.download(tickers,start='2005-01-01')
    df.to_parquet('datasets/sp100.parquet')
df = pd.read_parquet('datasets/sp100.parquet')
df

Adj Close                                                  \
                  AAPL        ABBV         ABT         ACN        ADBE   
Date                                                                     
2005-01-03    0.971844         NaN   14.956534   19.618639   30.838949   
2005-01-04    0.981825         NaN   14.805904   19.157375   30.024111   
2005-01-05    0.990424         NaN   14.636057   19.082981   29.859142   
2005-01-06    0.991192         NaN   14.966144   18.911869   29.364239   
2005-01-07    1.063362         NaN   15.222524   19.797201   29.384233   
...                ...         ...         ...         ...         ...   
2021-07-21  145.399994  116.989998  118.980003  313.769989  612.260010   
2021-07-22  146.800003  117.540001  118.250000  314.260010  623.679993   
2021-07-23  148.559998  118.190002  120.800003  318.649994  625.869995   
2021-07-26  148.990005  117.790001  119.519997  318.980011  620.799988   
2021-07-27  146.770004  117.959999  119.809998  319.890015  618.280029   

                                                                         ...  \
                   AIG        AMGN         AMT         AMZN        AVGO  ...   
Date                                                                     ...   
2005-01-03  873.705200   50.510906   15.225256    44.520000         NaN  ...   
2005-01-04  876.616089   49.709389   15.024044    42.139999         NaN  ...   
2005-01-05  891.171082   48.923599   14.889898    41.770000         NaN  ...   
2005-01-06  892.627014   49.379353   15.024044    41.049999         NaN  ...   
2005-01-07  894.214600   49.481529   15.292330    42.320000         NaN  ...   
...                ...         ...         ...          ...         ...  ...   
2021-07-21   48.290001  244.850006  280.260010  3585.199951  479.019989  ...   
2021-07-22   47.560001  244.740005  281.600006  3638.030029  476.329987  ...   
2021-07-23   47.310001  247.720001  285.260010  3656.639893  483.149994  ...   
2021-07-26   47.709999  245.080002  284.609985  3699.820068  482.239990  ...   
2021-07-27   47.419998  244.679993  286.880005  3626.389893  475.359985  ...   

             Volume                                                    \
                UNH      UNP       UPS      USB           V        VZ   
Date                                                                    
2005-01-03  6252600  7762400   2176500  5374600         NaN   9236061   
2005-01-04  5582000  8108400   3127400  4552700         NaN  11680234   
2005-01-05  4577600  5504800   2130700  5701600         NaN   8201616   
2005-01-06  4837600  5653600   1773600  4203000         NaN   7844458   
2005-01-07  6653200  3998000   2396900  5312700         NaN   8424450   
...             ...      ...       ...      ...         ...       ...   
2021-07-21  1863700  2320000   1770900  5544800  11241400.0  25264100   
2021-07-22  1421900  4194900   1629700  4042900  10293600.0  16231400   
2021-07-23  1611700  3946800   2188900  3570500   6762000.0  14874700   
2021-07-26  2232600  2840900   2669400  3721300   4875500.0  15328200   
2021-07-27  2630200  3199000  13695700  4332400   7034000.0  15224600   

                                                   
                WBA       WFC       WMT       XOM  
Date                                               
2005-01-03  9757800   8168200  16078400  16009200  
2005-01-04  6618200   9241400  10596700  16718000  
2005-01-05  6850200   7694600   8241500  12189100  
2005-01-06  4816400   8887600  14028800  13098700  
2005-01-07  2946400   6573200   9660300  11315300  
...             ...       ...       ...       ...  
2021-07-21  8544400  29654600   6197300  20908700  
2021-07-22  3519000  23309500   4337800  13493800  
2021-07-23  3742600  23944700   5314100  14081500  
2021-07-26  4055900  22945100   6171000  18374900  
2021-07-27  5119100  22117000   5130200  16846000  

[4170 rows x 606 columns]

In [42]:
prices = df['Adj Close']
prices.index = pd.to_datetime(prices.index)
monthly_rets = prices.pct_change().resample('M').agg(lambda x: (1+x).prod()-1)

In [43]:
formation_dt = dt.datetime(2009,2,28)
prior_monthly_rets = monthly_rets.loc[formation_dt -MonthEnd(1)]
prior_monthly_rets.name = 'monthly_rets'
prior_monthly_rets

AAPL    0.056005
ABBV    0.000000
ABT     0.046310
ACN    -0.037511
ADBE   -0.093001
          ...   
VZ     -0.106012
WBA     0.111066
WFC    -0.358888
WMT    -0.159472
XOM    -0.041964
Name: monthly_rets, Length: 101, dtype: float64

In [44]:
prior_monthly_rets = pd.DataFrame(prior_monthly_rets)
# Bucket returns
prior_monthly_rets['decile'] = pd.qcut(prior_monthly_rets.monthly_rets, 10, labels=False)
prior_monthly_rets

,monthly_rets,decile
AAPL,0.056005,8
ABBV,0.000000,7
ABT,0.046310,8
ACN,-0.037511,5
ADBE,-0.093001,3
...,...,...
VZ,-0.106012,2
WBA,0.111066,9
WFC,-0.358888,0
WMT,-0.159472,1


In [45]:
losers = prior_monthly_rets[prior_monthly_rets.decile == 0].index
losers_ret = monthly_rets.loc[formation_dt, monthly_rets.columns.isin(losers)].mean()
losers_ret

-0.2571558902685344

In [46]:
def monthly_dogs(formation_dt, monthly_rets):
    prior_monthly_rets = monthly_rets.loc[formation_dt -MonthEnd(1)]
    prior_monthly_rets.name = 'monthly_rets'
    prior_monthly_rets = pd.DataFrame(prior_monthly_rets)
    prior_monthly_rets['decile'] = pd.qcut(prior_monthly_rets.monthly_rets, 10, labels=False,duplicates='drop')
    losers = prior_monthly_rets[prior_monthly_rets.decile == 0].index
    losers_ret = monthly_rets.loc[formation_dt, monthly_rets.columns.isin(losers)].mean()
    return losers_ret
    #return (losers_ret, losers.values)

print('2009,2,28:', monthly_dogs(dt.datetime(2009,2,28), monthly_rets))
print(monthly_dogs(dt.datetime(2009,3,31), monthly_rets))



2009,2,28: -0.2571558902685344
0.3519540255828021


In [56]:
returns = []
dates = monthly_rets[2:].index
for dt in dates:
    returns.append(monthly_dogs(dt, monthly_rets))
df = pd.DataFrame({'dates': dates, 'returns': returns})
print(df.describe())
df

          returns
count  197.000000
mean     0.013746
std      0.066310
min     -0.257156
25%     -0.009985
50%      0.015704
75%      0.046138
max      0.351954


,dates,returns
0,2005-03-31,-0.042680
1,2005-04-30,-0.051364
2,2005-05-31,0.065292
3,2005-06-30,0.019790
4,2005-07-31,0.111726
...,...,...
192,2021-03-31,0.049651
193,2021-04-30,0.041486
194,2021-05-31,0.056212
195,2021-06-30,0.018880


In [66]:
#spy = yf.download('SPY',start='2005-01-01')
#spy = spy['Adj Close']
spy_rets = spy.pct_change().resample('M').agg(lambda x: (1+x).prod()-1)
df['spy_rets']  = spy_rets[2:].values
df.describe()

,returns,spy_rets
count,197.000000,197.000000
mean,0.013746,0.009140
std,0.066310,0.042181
min,-0.257156,-0.165187
25%,-0.009985,-0.013344
50%,0.015704,0.013567
75%,0.046138,0.032415
max,0.351954,0.126984
